In [82]:
import sys
import importlib

# Recargar el módulo data_utils para asegurar que se carguen los cambios
if 'src.data_utils' in sys.modules:
    importlib.reload(sys.modules['src.data_utils'])

# Carga de datos

Este notebook encapsula el proceso de carga de datos, transformación a series temporales y creación de características y target en funciones reutilizables para forecasting.

In [ ]:
from src.data_utils import load_raw_data # type: ignore

# Descarga y carga datos raw desde BigQuery para el rango de fechas especificado
df_raw = load_raw_data(
    fecha_inicio="2023-01-02", 
    fecha_fin="2025-06-30", 
    descargar_bq=False)

df_raw.describe()

Iniciando conexión con BigQuery...
Conexión establecida.
Ejecutando consulta SQL...
Conexión establecida.
Ejecutando consulta SQL...


c:\Workspace\mlops_fleca_project\.venv\lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Consulta finalizada. Filas descargadas: 337353
Guardando archivo en C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250630.parquet ...
Archivo guardado correctamente.
Cargando datos desde: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250630.parquet
Total de fechas faltantes: 4
Fechas faltantes: ['2023-12-25T00:00:00.000000000' '2024-01-01T00:00:00.000000000'
 '2024-12-25T00:00:00.000000000' '2025-01-01T00:00:00.000000000']


,fecha,cantidad,base_imponible,tipo_IVA,total,is_summer_peak,is_easter
count,337353,337353.000000,337353.000000,335106.000000,337353.000000,337353.000000,337353.0
mean,2024-03-10 00:54:30.032281344,1.209303,2.010886,9.084558,2.199180,0.208473,0.0
min,2023-01-02 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,2023-07-28 00:00:00,1.000000,1.350000,10.000000,1.450000,0.000000,0.0
50%,2024-03-09 00:00:00,1.000000,1.640000,10.000000,1.800000,0.000000,0.0
75%,2024-09-25 00:00:00,1.000000,2.320000,10.000000,2.550000,0.000000,0.0
max,2025-06-30 00:00:00,93.000000,287.270000,10.000000,316.000000,1.000000,0.0
std,NaN,0.733263,1.692278,2.793367,1.862156,0.406217,0.0


# Transformación a series temporales

In [84]:
from src.data_utils import transformar_a_series_temporales, guardar_time_series_interim # type: ignore

# Transformamos los datos raw a series temporales semanales para la familia BOLLERIA
df_familia_semanal = transformar_a_series_temporales(
    df_raw, 
    familia='BOLLERIA', 
    guardar_interim=True
)

# Visualiza las primeras filas
df_familia_semanal.head(150)

Archivo guardado en: C:\Workspace\mlops_fleca_project\data\interim\time_series_BOLLERIA_weekly_20250730.parquet


,year,week,familia,base_imponible,is_summer_peak,is_easter,dias_semana
0,2023,1,BOLLERIA,825.11,0,0,7
1,2023,2,BOLLERIA,658.40,0,0,7
2,2023,3,BOLLERIA,741.40,0,0,7
3,2023,4,BOLLERIA,653.64,0,0,7
4,2023,5,BOLLERIA,680.46,0,0,7
...,...,...,...,...,...,...,...
121,2025,22,BOLLERIA,802.16,0,0,7
122,2025,23,BOLLERIA,881.72,0,0,7
123,2025,24,BOLLERIA,1015.97,0,0,7
124,2025,25,BOLLERIA,1014.76,0,0,7


In [85]:
# Verificar los valores de is_easter en las fechas de Semana Santa
easter_dates = df_familia_semanal[df_familia_semanal['is_easter'] > 0]
print("Fechas con is_easter > 0:")
print(easter_dates[['year', 'week', 'is_easter']])

# Verificar que tenemos las 3 semanas de Semana Santa correctas
expected_easter_weeks = [
    (2023, 14),  # Semana Santa 2023
    (2024, 13),  # Semana Santa 2024
    (2025, 16)   # Semana Santa 2025
]

for year, week in expected_easter_weeks:
    is_marked = ((df_familia_semanal['year'] == year) & 
                 (df_familia_semanal['week'] == week) & 
                 (df_familia_semanal['is_easter'] == 1)).any()
    print(f"Semana {week} del {year} marcada como Semana Santa: {is_marked}")

Fechas con is_easter > 0:
     year  week  is_easter
13   2023    14          1
62   2024    13          1
115  2025    16          1
Semana 14 del 2023 marcada como Semana Santa: True
Semana 13 del 2024 marcada como Semana Santa: True
Semana 16 del 2025 marcada como Semana Santa: True


In [86]:
df_familia_semanal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            126 non-null    UInt32 
 1   week            126 non-null    UInt32 
 2   familia         126 non-null    object 
 3   base_imponible  126 non-null    float64
 4   is_summer_peak  126 non-null    int64  
 5   is_easter       126 non-null    int64  
 6   dias_semana     126 non-null    int64  
dtypes: UInt32(2), float64(1), int64(3), object(1)
memory usage: 6.3+ KB


# Transformación a features y target

In [87]:
from src.data_utils import transformar_features_target # type: ignore

# Demostración de generación de features y target para modelado
# Usamos la función centralizada que ya maneja los lags y prepara el target
X, y, df_completo = transformar_features_target(
    df_familia_semanal,
    lags_list=[1, 2, 3, 4, 12, 24, 52],
    columna_target='base_imponible',
    cols_exogenas=['is_summer_peak', 'is_easter'],
    eliminar_nulos=True
)

# Mostramos las dimensiones y primeras filas
print(f"Dimensión de X (features): {X.shape}")
print(f"Dimensión de y (target): {y.shape}")
print("\nPrimeras filas de X:")
X.head(200)

Dimensión de X (features): (73, 9)
Dimensión de y (target): (73,)

Primeras filas de X:


,base_imponible_lag1,base_imponible_lag2,base_imponible_lag3,base_imponible_lag4,base_imponible_lag12,base_imponible_lag24,base_imponible_lag52,is_summer_peak,is_easter
52,572.51,534.79,563.18,806.54,912.13,1745.97,825.11,0,0
53,597.65,572.51,534.79,563.18,750.99,1681.41,658.40,0,0
54,680.30,597.65,572.51,534.79,821.84,1753.02,741.40,0,0
55,603.99,680.30,597.65,572.51,607.02,1835.18,653.64,0,0
56,600.14,603.99,680.30,597.65,304.73,2127.71,680.46,0,0
...,...,...,...,...,...,...,...,...,...
120,810.97,842.62,891.38,894.20,725.72,756.58,828.30,0,0
121,756.42,810.97,842.62,891.38,635.99,604.75,854.34,0,0
122,802.16,756.42,810.97,842.62,673.60,636.84,782.38,0,0
123,881.72,802.16,756.42,810.97,546.93,571.22,708.74,0,0


In [88]:
# Verificar que las semanas de Semana Santa están correctamente representadas en el conjunto de datos final
easter_weeks_in_final = df_completo[df_completo['is_easter'] == 1]
print("Semanas de Semana Santa en el conjunto final de datos:")
print(easter_weeks_in_final[['is_easter', 'is_summer_peak', 'base_imponible_lag1', 'base_imponible_next1']])

Semanas de Semana Santa en el conjunto final de datos:
     is_easter  is_summer_peak  base_imponible_lag1  base_imponible_next1
62           1               0               706.42                823.62
115          1               0               620.41                894.20


In [89]:
# Verificar por qué no aparece la Semana Santa de 2023
# Buscar la fila correspondiente en df_completo (antes de filtrar nulos)
ss_2023 = df_completo[(df_completo['year'] == 2023) & (df_completo['week'] == 14)]
print("Semana Santa 2023 antes de filtrar nulos:")
print(ss_2023[['year', 'week', 'is_easter', 'base_imponible_lag52']])
print("\nLa Semana Santa de 2023 (semana 14) no tiene datos de lag52 porque no hay datos de 2022")

Semana Santa 2023 antes de filtrar nulos:
Empty DataFrame
Columns: [year, week, is_easter, base_imponible_lag52]
Index: []

La Semana Santa de 2023 (semana 14) no tiene datos de lag52 porque no hay datos de 2022


# Guardado de datos procesados

Vamos a guardar los datasets finales (X, y, df_completo) en la carpeta `processed` ya que estos datos están listos para ser utilizados en el modelado.

In [90]:
# Importar la nueva función para guardar datos procesados
from src.data_utils import guardar_datos_procesados # type: ignore

# Guardar los datasets en la carpeta processed
archivos_guardados = guardar_datos_procesados(
    X=X, 
    y=y, 
    df_completo=df_completo,
    familia='BOLLERIA'
)

# Verificar las rutas donde se guardaron los archivos
for tipo_datos, ruta in archivos_guardados.items():
    print(f"Archivo {tipo_datos}: {ruta}")

Datos procesados guardados en la carpeta: C:\Workspace\mlops_fleca_project\data\processed
- Features (X): ts_X_bolleria_20250730.parquet
- Target (y): ts_y_bolleria_20250730.parquet
- Dataset completo: ts_df_bolleria_20250730.parquet
Archivo X: C:\Workspace\mlops_fleca_project\data\processed\ts_X_bolleria_20250730.parquet
Archivo y: C:\Workspace\mlops_fleca_project\data\processed\ts_y_bolleria_20250730.parquet
Archivo df_completo: C:\Workspace\mlops_fleca_project\data\processed\ts_df_bolleria_20250730.parquet


# Conclusiones sobre la organización de los datos

Siguiendo las mejores prácticas de proyectos MLOps, hemos organizado nuestros datos de la siguiente manera:

1. **Datos Raw** (`data/raw`): Datos originales descargados de BigQuery, sin procesar.

2. **Datos Interim** (`data/interim`): Datos intermedios que han pasado por alguna transformación pero aún no están listos para modelado.
   - Las series temporales semanales generadas por `transformar_a_series_temporales()` se guardan aquí.

3. **Datos Processed** (`data/processed`): Datos finales listos para alimentar modelos de machine learning.
   - Los datasets de features (X), target (y) y dataset completo generados por `transformar_features_target()` se guardan aquí.

Esta organización nos permite tener una trazabilidad clara del flujo de datos y facilita el trabajo colaborativo.